<a href="https://colab.research.google.com/github/Droslj/scATAC-seq-complete-/blob/Google-colab/scATAC_seq_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scATAC seq, based on scATAC seq processing Galaxy tutorials (scATAC preprocessing (2), Standard scATAC seq processing pipeline (1) )
AD Objects created in Galaxy using customized Galaxy WF with Snapatac2 and imported
(1) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-preprocessing-tenx/tutorial.html#mapping-reads-to-a-reference-genome, (2) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-standard-processing-snapatac2/tutorial.html
Data taken from the following NCBI study:
Metabolic adaptation pilots the differentiation of human hematopoietic cells (https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1015713)
Import Anndata objects for two biological replicates, SRR26046013 (cells treated with AOA inhibitor) and SRR26046019 (untreated cells)
Perform following steps:
(1) Import matrices
(2) Compute fragment size distribution
(3) Compute TSS enrichment
(4) Filter cell counts based on TSSe
(5) Create cell by bin matrix based on 500 bp wide bins accross the whole genome
(6) Perform feature selection
(7) Perform Doublet removal
(8) Perform Dim reduction (spectral)
(9) Perform Clustering (neighborhood, UMAP, leiden)
(10) Create a cell by gene matrix
(11) Concatenate matrices using Inner join
(12) Remove batch effects

In [1]:
!pip install -q condacolab

In [2]:
import condacolab

In [3]:
condacolab.install()

✨🍰✨ Everything looks OK!


In [4]:
!conda --version

conda 23.11.0


In [5]:
!which conda

/usr/local/bin/conda


In [6]:
!conda config --add channels conda-forge

In [7]:
!conda config --add channels bioconda

In [11]:
!pip install snapatac2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00


In [12]:
!pip show snapatac2

Name: snapatac2
Version: 2.7.1
Summary: SnapATAC2: Single-cell epigenomics analysis pipeline
Home-page: https://github.com/
Author: Kai Zhang <kai@kzhang.org>
Author-email: Kai Zhang <zhangkai33@westlake.edu.cn>
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, igraph, kaleido, macs3, multiprocess, natsort, numpy, pandas, plotly, polars, pooch, pyarrow, pyfaidx, rustworkx, scikit-learn, scipy, tqdm, typeguard
Required-by: 


In [13]:
import snapatac2 as snap

In [14]:
!pip install umap-learn -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 56.9 MB/s eta 0:00:00


In [15]:
import umap.umap_ as umap


In [16]:
from umap import UMAP

In [17]:
!pip install scanpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━

In [18]:
import scanpy as sc

In [19]:
pip show scanpy

Name: scanpy
Version: 1.10.4
Summary: Single-Cell Analysis in Python.
Home-page: 
Author: Alex Wolf, Philipp Angerer, Fidel Ramirez, Isaac Virshup, Sergei Rybakov, Gokcen Eraslan, Tom White, Malte Luecken, Davide Cittaro, Tobias Callies, Marius Lange, Andrés R. Muñoz-Rojas
Author-email: 
License: 
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, h5py, joblib, legacy-api-wrap, matplotlib, natsort, networkx, numba, numpy, packaging, pandas, patsy, pynndescent, scikit-learn, scipy, seaborn, session-info, statsmodels, tqdm, umap-learn
Required-by: 


In [20]:
!pip install magic-impute -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.6 MB/s eta 0:00:00


In [22]:
!pip install leidenalg -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.4 MB/s eta 0:00:00


In [21]:
import numpy as np

In [23]:
import anndata as ad

In [24]:
import matplotlib.pyplot as plt

In [26]:
import plotly.subplots as sp
import plotly.graph_objects as go

# Import reads from google drive, one sample treated with energy metabolism inhibitors and one untreated

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
SRR26046013_DM_AOA_INH = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/SRR26046013_Annotated_data_matrix.h5ad')

In [28]:
SRR26046013_DM_AOA_INH

AnnData object with n_obs × n_vars = 13546 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

In [29]:
SRR26046019_DM_UT = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/SRR26046019_Annotated_data_matrix.h5ad')

In [30]:
SRR26046019_DM_UT

AnnData object with n_obs × n_vars = 10448 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

Compute fragment size distributions

In [ ]:
# Create a subplot figure
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Plot for SRR26046013_DM_AOA_INH
frag_size_distr = snap.metrics.frag_size_distr(SRR26046013_DM_AOA_INH, inplace=False)
axes[0].plot(frag_size_distr)
axes[0].set_title('Treated w/AOA')

# Plot for SRR26046019_DM_UT
frag_size_distr4 = snap.metrics.frag_size_distr(SRR26046019_DM_UT, inplace=False)
axes[1].plot(frag_size_distr4)
axes[1].set_title('Untreated')

plt.tight_layout()
plt.show()

# Compute and plot TSSe

In [ ]:
# Compute TSSe metrics
# Get genome annotation
gene_anno = snap.genome.hg38

In [ ]:
snap.metrics.tsse(SRR26046013_DM_AOA_INH, gene_anno)

In [ ]:
snap.metrics.tsse(SRR26046019_DM_UT, gene_anno)

In [ ]:
# Generate TSSE plots
TSSE1plot = snap.pl.tsse(SRR26046013_DM_AOA_INH, show = False)
TSSE2plot = snap.pl.tsse(SRR26046019_DM_UT,show = False)

# Create a subplot figure
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Treated w/AOA', 'Untreated'))

# Add the plots to the subplot figure
fig.add_trace(TSSE1plot.data[0], row=1, col=1)
fig.add_trace(TSSE2plot.data[0], row=1, col=2)

# Update layout and set X-axis to logarithmic scale
fig.update_layout(height=400, width=800, title_text="TSS enrichment")
fig.update_xaxes(type="log", row=1, col=1)
fig.update_xaxes(type="log", row=1, col=2)

fig.show()
fig.write_image("TSSE_plots.png", height=1080, width=1920)

In [ ]:
gene_anno

fig.write_image("TSSe_plot.png",  width=1920, height=1080)

#Filter cell counts and TSSE values based on above plots

In [ ]:
snap.pp.filter_cells(SRR26046013_DM_AOA_INH, min_counts=3000, min_tsse=15, max_counts=70000)

In [ ]:
snap.pp.filter_cells(SRR26046019_DM_UT, min_counts=6000, min_tsse=13, max_counts=90000)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046019_DM_UT

Write to file
SRR26046013_DM_AOA_INH.write_h5ad('SRR26046013_DM_AOA_INH_filt.h5ad')
SRR26046019_DM_UT.write_h5ad('SRR26046018_DM_UT_filt.h5ad')

# Create cell by bin matrix containing insertion counts across genome-wide 500-bp bins

In [ ]:
snap.pp.add_tile_matrix(SRR26046013_DM_AOA_INH)

In [ ]:
snap.pp.add_tile_matrix(SRR26046019_DM_UT)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046019_DM_UT

# Perform feature selection

In [ ]:
snap.pp.select_features(SRR26046013_DM_AOA_INH, n_features = 250000)

In [ ]:
snap.pp.select_features(SRR26046019_DM_UT, n_features = 250000)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
#sc.write(adata=SRR26046013_DM_AOA_INH,filename="SRR26046013_Annotated_data_matrix_P1.h5ad",compression='gzip')

# Doublet removal

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
#Apply a scrublet algorithm to identify potential doublets
snap.pp.scrublet(SRR26046013_DM_AOA_INH)

In [ ]:
snap.pp.scrublet(SRR26046019_DM_UT)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046019_DM_UT

In [ ]:
#Filter doublets
snap.pp.filter_doublets(SRR26046013_DM_AOA_INH)

In [ ]:
snap.pp.filter_doublets(SRR26046019_DM_UT)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046019_DM_UT

# Dimension reduction

In [ ]:
snap.tl.spectral(SRR26046013_DM_AOA_INH)

In [ ]:
snap.tl.spectral(SRR26046019_DM_UT)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046019_DM_UT

In [ ]:
snap.tl.umap(SRR26046013_DM_AOA_INH)

In [ ]:
snap.tl.umap(SRR26046019_DM_UT)

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046019_DM_UT

# Clustering analysis

In [ ]:
#(1)Calculate knn graph, (2) use Leiden community detection, (3) plot UMAP (returns plotly object)

In [ ]:
snap.pp.knn(SRR26046013_DM_AOA_INH)
snap.tl.leiden(SRR26046013_DM_AOA_INH)
SRR26046013_DM_AOA_INH_UMAP = snap.pl.umap(SRR26046013_DM_AOA_INH, color = 'leiden', interactive = False, show = False)

In [ ]:
snap.pp.knn(SRR26046019_DM_UT)
snap.tl.leiden(SRR26046019_DM_UT)
SRR26046019_DM_UT_UMAP = snap.pl.umap(SRR26046019_DM_UT, color='leiden', interactive=False, show=False)

In [ ]:
# Create a subplot figure
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Treated w/AOA', 'Untreated'))

# Add all traces from each UMAP plot to the subplot figure
for trace in SRR26046013_DM_AOA_INH_UMAP.data:
 trace.showlegend = False  # Hide legend for individual traces
 fig.add_trace(trace, row=1, col=1)

for trace in SRR26046019_DM_UT_UMAP.data:
 fig.add_trace(trace, row=1, col=2)  # Show legend only for the last subplot

# Update layout with legend title and position
fig.update_layout(height=600, width=1200, title_text="UMAP Clustering", legend_title_text='Clusters', legend=dict(x=1.05, y=1, traceorder='normal', font=dict(family='sans-serif', size=12, color='black'), bordercolor='Black', borderwidth=1))

fig.show()
fig.write_image("Cluster_plots.png", height=1080, width=1920)

Write structures to file
SRR26046013_DM_AOA_INH.write_h5ad("SRR26046013_PI.h5ad")

SRR26046019_DM_UT.write_h5ad("SRR26046019_PI.h5ad")

Create a cell-by-gene activity matrix

In [ ]:
#sc.write(SRR26046013_DM_AOA_INH,"/SRR26046013_DM_AOA_INH.h5ad")

In [ ]:
SRR26046019_DM_UT

In [ ]:
SRR26046013_DM_AOA_INH_GM = snap.pp.make_gene_matrix(SRR26046013_DM_AOA_INH, snap.genome.hg38)

In [ ]:
SRR26046019_DM_UT_GM = snap.pp.make_gene_matrix(SRR26046019_DM_UT, snap.genome.hg38)

In [ ]:
SRR26046013_DM_AOA_INH_GM

In [ ]:
SRR26046019_DM_UT_GM

In [ ]:
SRR26046013_DM_AOA_INH

In [ ]:
SRR26046013_DM_AOA_INH_GM

In [ ]:
SRR26046019_DM_UT_GM

# Imputation with scanpy and MAGIC

In [ ]:
%%time
sc.pp.filter_genes(SRR26046013_DM_AOA_INH_GM, min_cells= 5)
sc.pp.normalize_total(SRR26046013_DM_AOA_INH_GM)
sc.pp.log1p(SRR26046013_DM_AOA_INH_GM)



In [ ]:
sc.external.pp.magic(SRR26046013_DM_AOA_INH_GM, solver="approximate")

In [ ]:
# Copy over UMAP embedding
SRR26046013_DM_AOA_INH_GM.obsm["X_umap"] = SRR26046013_DM_AOA_INH.obsm["X_umap"]

In [ ]:
sc.pp.filter_genes(SRR26046019_DM_UT_GM, min_cells= 5)
sc.pp.normalize_total(SRR26046019_DM_UT_GM)
sc.pp.log1p(SRR26046019_DM_UT_GM)

In [ ]:
sc.external.pp.magic(SRR26046019_DM_UT_GM, solver="approximate")

In [ ]:
# Copy over UMAP embedding
SRR26046019_DM_UT_GM.obsm["X_umap"] = SRR26046019_DM_UT.obsm["X_umap"]

In [ ]:
SRR26046013_DM_AOA_INH_GM

In [ ]:
SRR26046019_DM_UT_GM

# Preprocess according to scanpy flow

In [ ]:
sc.pp.scale(SRR26046013_DM_AOA_INH_GM, max_value = 10)
sc.pp.pca(SRR26046013_DM_AOA_INH_GM, svd_solver = 'arpack')
sc.pp.neighbors(SRR26046013_DM_AOA_INH_GM, n_neighbors=10, n_pcs=40)
sc.tl.umap(SRR26046013_DM_AOA_INH_GM)
sc.tl.leiden(SRR26046013_DM_AOA_INH_GM)

In [ ]:
%%time
sc.pp.scale(SRR26046019_DM_UT_GM, max_value = 10)
sc.pp.pca(SRR26046019_DM_UT_GM, svd_solver = 'arpack')
sc.pp.neighbors(SRR26046019_DM_UT_GM, n_neighbors=10, n_pcs=40)
sc.tl.umap(SRR26046019_DM_UT_GM)

In [ ]:
sc.tl.leiden(SRR26046019_DM_UT_GM)

In [ ]:
SRR26046013_DM_AOA_INH_GM

In [ ]:
SRR26046019_DM_UT_GM

# Identify marker genes

In [ ]:
SRR26046013_DM_AOA_INH_GM_MG = sc.tl.rank_genes_groups(SRR26046013_DM_AOA_INH_GM, 'leiden', method='wilcoxon', copy = True)
SRR26046018_DM_UT_GM_MG = sc.tl.rank_genes_groups(SRR26046019_DM_UT_GM, 'leiden', method='wilcoxon', copy = True)

In [ ]:
sc.pl.pca_overview(SRR26046013_DM_AOA_INH_GM, color ='leiden')

In [ ]:
sc.pl.pca_overview(SRR26046019_DM_UT_GM, color ='leiden')

In [ ]:
#Obtain ranked genes for all clustes
ranked_genes_dict = SRR26046013_DM_AOA_INH_GM_MG.uns['rank_genes_groups']

# Get all cluster IDs
cluster_ids = ranked_genes_dict['names'].dtype.names

# Iterate through each cluster and print the top 10 genes
for cluster_id in cluster_ids:
    # Get ranked genes for the current cluster
    cluster_genes = ranked_genes_dict['names'][cluster_id]

    # Print the top 10 genes for the current cluster
    print(f"Top 10 genes for cluster {cluster_id}:")
    print(cluster_genes[:10])
    print("\n")

# Concatenate Data matrices

In [ ]:
#Use inner join
adata_concat = ad.concat([SRR26046013_DM_AOA_INH_GM, SRR26046019_DM_UT_GM], label = 'Treatment', keys = ['Treated w/AOA', 'Untreated'], join='inner')

# Remove batch effects

In [ ]:
import scanpy.external as sce

In [ ]:
pip install harmonypy -q

In [ ]:
import harmonypy as hm

In [ ]:
import seaborn as sns

In [ ]:
sce.pp.harmony_integrate(adata_concat, 'Treatment')

In [ ]:
adata_concat

Plot the 'obsm' before and after batch effect removal (PC1 vs PC2)

In [ ]:
#Extract data sets
x_before = adata_concat.obsm['X_pca'][:, 0]
y_before = adata_concat.obsm['X_pca'][:, 1]

x_after = adata_concat.obsm['X_pca_harmony'][:, 0]
y_after = adata_concat.obsm['X_pca_harmony'][:, 1]
batch_labels = adata_concat.obs['Treatment']


In [ ]:
#Define figure
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
#Plot before batch effect removal

sns.scatterplot(x=x_before, y=y_before, hue=batch_labels)
plt.title('PCA Before Batch Effect Removal')
plt.xlabel('PC1')
plt.ylabel('PC2')

#Plot PCA after batch effect removal

plt.subplot(1, 2, 2)
sns.scatterplot(x=x_after, y=y_after, hue=batch_labels)
plt.title('PCA After Batch Effect Removal')
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.tight_layout()
plt.show()

In [ ]:
ad.AnnData.write_h5ad(adata_concat, 'adata_concat.h5ad')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from shutil import copyfile

In [ ]:
copyfile('adata_concat.h5ad', '/content/drive/MyDrive/Colab Notebooks/adata_concat.h5ad')

In [ ]:
adata_concat

In [ ]:
!pip install pydeseq2  # Install PyDESeq2
!pip install pandas #Install Pandas

In [ ]:
import scanpy as sc
import snapatac2 as snap

# Create cell-by-gene matrix (replace with cell-by-bin if appropriate)
gene_matrix_1 = snap.pp.make_gene_matrix(your_adata_object_1, snap.genome.hg38)
gene_matrix_2 = snap.pp.make_gene_matrix(your_adata_object_2, snap.genome.hg38)

# Extract count matrices and convert to pandas DataFrames
count_matrix_1 = gene_matrix_1.X.toarray()  # Assuming count data is in .X
count_matrix_2 = gene_matrix_2.X.toarray()
count_df_1 = pd.DataFrame(count_matrix_1, index=gene_matrix_1.obs_names, columns=gene_matrix_1.var_names)
count_df_2 = pd.DataFrame(count_matrix_2, index=gene_matrix_2.obs_names, columns=gene_matrix_2.var_names)

In [ ]:
metadata_df = pd.DataFrame(index=count_df.index)
metadata_df['treatment'] = ['condition1'] * len(your_adata_object_1.obs_names) + ['condition2'] * len(your_adata_object_2.obs_names)  # Replace with your conditions
# ... add other metadata variables as needed

In [ ]:
import pydeseq2

# Combine the two scATAC-seq count matrices
count_df = pd.concat([count_df_1, count_df_2], axis=0)

# Construct a DESeqDataSet object
dds = pydeseq2.DESeqDataSet(count_df, metadata_df, design_formula='~ treatment')  # Adjust design formula

# Perform differential accessibility analysis
dds.run_deseq()
dds.get_results()